# N-Puzzle Problem

In [33]:
import numpy as np
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic

In [34]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

# Here we are going to check the available actions for the current state
def available_actions(state: np.array) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x,y), (x-1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x,y), (x+1, y)))
    if y > 0:
        actions.append(action((x,y), (x, y-1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x,y), (x, y+1)))
    return actions
    
# Here we are going to move the empty tile to the new position
def move(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state


# Here we are going to check if the state is solved
def checkIfSolved(finalState: np.ndarray) -> bool:
    return np.array_equal(finalState, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape(PUZZLE_DIM, PUZZLE_DIM))
    

### Ensuring Solvable Puzzle States with Inversions Check
This code checks if a randomized puzzle state is solvable by counting inversions. If the state is unsolvable, it further randomizes the puzzle up to ten attempts to achieve a solvable state.



In [35]:
# Here we define the number of steps to randomize the puzzle (goal state)
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape(PUZZLE_DIM, PUZZLE_DIM)
goal_state = state.copy()

# Here we count the number of inversions in the state to check afterwards if it is solvable
def count_inversions(state: np.ndarray) -> int:
    flat_state = state.flatten()
    flat_state = flat_state[flat_state != 0]  # Exclude the blank tile
    inversions = 0
    for i in range(len(flat_state)):
        for j in range(i + 1, len(flat_state)):
            if flat_state[i] > flat_state[j]:
                inversions += 1
    return inversions

# Here we check if the state is solvable by checking the number of inversions and the position of the blank tile
def is_solvable(state: np.ndarray) -> bool:
    inversions = count_inversions(state)
    blank_row = np.where(state == 0)[0][0]  # Zero-based row number from top
    blank_row_from_bottom = PUZZLE_DIM - blank_row

    if PUZZLE_DIM % 2 == 1:  # Odd grid size (e.g., 3x3)
        return inversions % 2 == 0
    else:  # Even grid size (e.g., 4x4)
        if blank_row_from_bottom % 2 == 0:
            return inversions % 2 == 1
        else:
            return inversions % 2 == 0

# Here we keep randomizing until we get a solvable state
attempts = 0
max_attempts = 10
initial_state = None

while attempts < max_attempts:
    # We are going to randomize the goal state
    current_state = goal_state.copy()
    
    # Do shorter randomization steps
    steps = RANDOMIZE_STEPS // 5 if attempts > 0 else RANDOMIZE_STEPS
    for r in tqdm(range(steps), desc=f'Randomizing (Attempt {attempts + 1})'):
        current_state = move(current_state, choice(available_actions(current_state)))
    
    # Check if this randomized state is solvable
    if is_solvable(current_state):
        initial_state = current_state.copy()
        print("\nFound a solvable initial state!")
        print(f"Number of inversions: {count_inversions(initial_state)}")
        print(f"Blank tile row from bottom: {PUZZLE_DIM - np.where(initial_state == 0)[0][0]}")
        print('Initial State:')
        print(initial_state)
        print('\nGoal State:')
        print(goal_state)
        break
    
    print(f"\nAttempt {attempts + 1} generated an unsolvable state.")
    attempts += 1

if initial_state is None:
    raise Exception("Failed to find a solvable state after maximum attempts. Please try running the code again.")

Randomizing (Attempt 1): 100%|██████████| 100000/100000 [00:00<00:00, 105683.70it/s]


Found a solvable initial state!
Number of inversions: 10
Blank tile row from bottom: 2
Initial State:
[[4 1 2]
 [7 0 6]
 [8 3 5]]

Goal State:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


## Using different path search algorithms to solve the n^2-1 puzzle problem!

In [36]:
# Function to print the solution path
def print_solution_path_with_metrics(initial_state, solution_path):
    state = initial_state.copy()
    print("Initial State:")
    print(state)
    print("\nSteps to solve:")

    for i, action in enumerate(solution_path, start=1):
        print(f"\nStep {i}: Move tile from {action.pos2} to {action.pos1}")
        state = move(state, action)
        print(state)

    print("\nGoal State Reached!")


### BFS Algorithm

In [37]:
from collections import deque
# Function to perform Breadth First Search
def bfs(initial_state):
    frontier = deque([(initial_state, [])])  # here we initialize the frontier with the initial state [] and empty path
    visited = set()  # We save the visited states here as a set because it is faster to check if a state is in a set than in a list
    
    # Here we start the BFS algorithm and with steps we count the number of iterations (or levels in the tree)
    steps = 0

    while frontier: # while the frontier is not empty (we still have states to explore)
        current_state, path = frontier.popleft()  # Dequeue the next state to explore
        steps += 1 # Increment step counter for each explored state. 

        # Here we check if the current state is the goal state
        if checkIfSolved(current_state):
            quality = len(path)
            print(f"Solution Quality: {quality} actions")
            print(f"Total Cost: {steps} states evaluated")
            print(f"Efficiency: {quality / steps:.4f}")
            return path  # Return the sequence of actions to reach the goal

        # Add the current state to the visited set
        visited.add(current_state.tobytes()) # We use tobytes() to convert the numpy array to a hashable object 

        # Generate all possible actions (moves) from the current state
        for action in available_actions(current_state):
            next_state = move(current_state, action) # apply the action to the current state to get the next state

            # Only enqueue states we haven't visited
            if next_state.tobytes() not in visited:
                frontier.append((next_state, path + [action])) # Enqueue the next state and the path to reach it

    print("No solution found.")
    return None


In [38]:
solution_path = bfs(initial_state)
if solution_path:
    print_solution_path_with_metrics(initial_state, solution_path)


Solution Quality: 12 actions
Total Cost: 2062 states evaluated
Efficiency: 0.0058
Initial State:
[[4 1 2]
 [7 0 6]
 [8 3 5]]

Steps to solve:

Step 1: Move tile from (2, 1) to (1, 1)
[[4 1 2]
 [7 3 6]
 [8 0 5]]

Step 2: Move tile from (2, 2) to (2, 1)
[[4 1 2]
 [7 3 6]
 [8 5 0]]

Step 3: Move tile from (1, 2) to (2, 2)
[[4 1 2]
 [7 3 0]
 [8 5 6]]

Step 4: Move tile from (1, 1) to (1, 2)
[[4 1 2]
 [7 0 3]
 [8 5 6]]

Step 5: Move tile from (2, 1) to (1, 1)
[[4 1 2]
 [7 5 3]
 [8 0 6]]

Step 6: Move tile from (2, 0) to (2, 1)
[[4 1 2]
 [7 5 3]
 [0 8 6]]

Step 7: Move tile from (1, 0) to (2, 0)
[[4 1 2]
 [0 5 3]
 [7 8 6]]

Step 8: Move tile from (0, 0) to (1, 0)
[[0 1 2]
 [4 5 3]
 [7 8 6]]

Step 9: Move tile from (0, 1) to (0, 0)
[[1 0 2]
 [4 5 3]
 [7 8 6]]

Step 10: Move tile from (0, 2) to (0, 1)
[[1 2 0]
 [4 5 3]
 [7 8 6]]

Step 11: Move tile from (1, 2) to (0, 2)
[[1 2 3]
 [4 5 0]
 [7 8 6]]

Step 12: Move tile from (2, 2) to (1, 2)
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Goal State Reached!


### DFS Algorithm

In [39]:
def dfs(initial_state, max_depth=60):
    frontier = [(initial_state, [])]  # (current state, path of actions leading to state)
    visited = set()  # To track visited states
    
    # Track number of steps to reach the solution
    steps = 0

    while frontier:
        current_state, path = frontier.pop()  # Pop the last state from the stack
        steps += 1

        # Check if the current state matches the goal state
        if checkIfSolved(current_state):
            quality = len(path)
            print(f"Solution Quality: {quality} actions")
            print(f"Total Cost: {steps} states evaluated")
            print(f"Efficiency: {quality / steps:.4f}")
            return path

        # Add the current state to the visited set
        visited.add(current_state.tobytes())

        # Generate all possible actions (moves) from the current state
        for action in available_actions(current_state):
            next_state = move(current_state, action)

            # Only push states we haven't visited and are within depth limits
            if next_state.tobytes() not in visited and len(path) < max_depth:
                frontier.append((next_state, path + [action]))

    print("No solution found.")
    return None


In [44]:
dfs_solution_path = dfs(initial_state)
if dfs_solution_path:
    print_solution_path_with_metrics(initial_state, dfs_solution_path)
print(dfs_solution_path)

Solution Quality: 50 actions
Total Cost: 138444 states evaluated
Efficiency: 0.0004
Initial State:
[[4 1 2]
 [7 0 6]
 [8 3 5]]

Steps to solve:

Step 1: Move tile from (1, 2) to (1, 1)
[[4 1 2]
 [7 6 0]
 [8 3 5]]

Step 2: Move tile from (2, 2) to (1, 2)
[[4 1 2]
 [7 6 5]
 [8 3 0]]

Step 3: Move tile from (2, 1) to (2, 2)
[[4 1 2]
 [7 6 5]
 [8 0 3]]

Step 4: Move tile from (2, 0) to (2, 1)
[[4 1 2]
 [7 6 5]
 [0 8 3]]

Step 5: Move tile from (1, 0) to (2, 0)
[[4 1 2]
 [0 6 5]
 [7 8 3]]

Step 6: Move tile from (1, 1) to (1, 0)
[[4 1 2]
 [6 0 5]
 [7 8 3]]

Step 7: Move tile from (1, 2) to (1, 1)
[[4 1 2]
 [6 5 0]
 [7 8 3]]

Step 8: Move tile from (0, 2) to (1, 2)
[[4 1 0]
 [6 5 2]
 [7 8 3]]

Step 9: Move tile from (0, 1) to (0, 2)
[[4 0 1]
 [6 5 2]
 [7 8 3]]

Step 10: Move tile from (0, 0) to (0, 1)
[[0 4 1]
 [6 5 2]
 [7 8 3]]

Step 11: Move tile from (1, 0) to (0, 0)
[[6 4 1]
 [0 5 2]
 [7 8 3]]

Step 12: Move tile from (1, 1) to (1, 0)
[[6 4 1]
 [5 0 2]
 [7 8 3]]

Step 13: Move tile from 

### A* Algorithm

### What is A* Algo? 
A* is a search algorithm that uses both:

Cost to reach a node (g(n)): The actual cost incurred from the start node to the current node.
Estimated cost to the goal (h(n)): A heuristic estimate of the cost to reach the goal from the current node.
It combines these into a total cost function: 
f(n)=g(n)+h(n) Where:
g(n): Tracks the exact path cost so far.
h(n): Uses a heuristic to estimate the remaining cost. f(n): Total estimated cost of the path through the node.
A* prioritizes nodes with the smallest f(n) value, ensuring that it finds the optimal solution if h(n) is admissible (doesn’t overestimate).



In [41]:
# We use manhatten distance as our heuristic function. 
# Manhatten distance is the sum of the distances of each tile from its goal position.
def manhatten_distance(state, goal_state):
    distance = 0
    for i in range(PUZZLE_DIM):
        for j in range(PUZZLE_DIM):
            if state[i][j] != 0:
                x, y = np.where(goal_state == state[i][j])
                distance += abs(x[0] - i) + abs(y[0] - j)
    return distance

In [42]:
import heapq

def A_star_algo(initial_state, goal_state):
    # priority queue stores (f(n), g(n), state, path)
    class State: 
        def __init__(self, state, g, path):
            self.state = state # Current state of the puzzle
            self.g = g # Cost from the initial state to the current state (g(n))
            self.h = manhatten_distance(state, goal_state) # Heuristic cost from current state to goal state (h(n))
            self.f = self.g + self.h # Total cost of the current state (f(n)) = g(n) + h(n)
            self.path = path # Path from the initial state to the current state
        
        # Python's heapq needs a way to compare objects. By defining __lt__ we can compare two states to store them in the priority queue
        def __lt__(self, other): # Less than function to compare two states
            if self.f == other.f: # if the total cost is the same, we compare the heuristic cost
                return self.f < other.f 
            return self.f < other.f
        
    
    # Initialize the frontier with the initial state
    initial_state = State(initial_state, 0, []) 
    frontier = [] # priority queue of states to be explored
    visited = set() # states already evaluated. (visited nodes)

    
    # we use heapq to implement the priority queue. we implement a min heap by storing the states with the lowest f(n) on top
    heapq.heappush(frontier, initial_state) # push the initial state to the priority queue 
    nodes_expanded = 0
    
    while frontier:
        current = heapq.heappop(frontier) # removes and returns the state with the lowest f(n) from the priority queue
        nodes_expanded += 1 
        # Check if we've reached the goal
        if checkIfSolved(current.state):
            quality = len(current.path)
            print(f"Solution Quality: {quality} actions")
            print(f"Total Cost: {nodes_expanded} states evaluated")
            print(f"Efficiency: {quality / nodes_expanded:.4f}")
            return current.path
            
        # Skip if already explored
        state_bytes = current.state.tobytes()
        if state_bytes in visited:
            continue
            
        visited.add(state_bytes)
        
        # generate all possible actions(states) from the current state and add them to the frontier 
        for next_action in available_actions(current.state):
            next_state = move(current.state, next_action)
            
            # Skip if already explored
            if next_state.tobytes() in visited:
                continue
                
            new_state = State(
                next_state,
                current.g + 1,
                current.path + [next_action]
            )
            
            heapq.heappush(frontier, new_state)
    
    print("No solution found!")
    return None

    
            
    

In [43]:
# Usage example:
solution_path = A_star_algo(initial_state, goal_state)
if solution_path:
    print_solution_path_with_metrics(initial_state, solution_path)

Solution Quality: 12 actions
Total Cost: 42 states evaluated
Efficiency: 0.2857
Initial State:
[[4 1 2]
 [7 0 6]
 [8 3 5]]

Steps to solve:

Step 1: Move tile from (2, 1) to (1, 1)
[[4 1 2]
 [7 3 6]
 [8 0 5]]

Step 2: Move tile from (2, 2) to (2, 1)
[[4 1 2]
 [7 3 6]
 [8 5 0]]

Step 3: Move tile from (1, 2) to (2, 2)
[[4 1 2]
 [7 3 0]
 [8 5 6]]

Step 4: Move tile from (1, 1) to (1, 2)
[[4 1 2]
 [7 0 3]
 [8 5 6]]

Step 5: Move tile from (2, 1) to (1, 1)
[[4 1 2]
 [7 5 3]
 [8 0 6]]

Step 6: Move tile from (2, 0) to (2, 1)
[[4 1 2]
 [7 5 3]
 [0 8 6]]

Step 7: Move tile from (1, 0) to (2, 0)
[[4 1 2]
 [0 5 3]
 [7 8 6]]

Step 8: Move tile from (0, 0) to (1, 0)
[[0 1 2]
 [4 5 3]
 [7 8 6]]

Step 9: Move tile from (0, 1) to (0, 0)
[[1 0 2]
 [4 5 3]
 [7 8 6]]

Step 10: Move tile from (0, 2) to (0, 1)
[[1 2 0]
 [4 5 3]
 [7 8 6]]

Step 11: Move tile from (1, 2) to (0, 2)
[[1 2 3]
 [4 5 0]
 [7 8 6]]

Step 12: Move tile from (2, 2) to (1, 2)
[[1 2 3]
 [4 5 6]
 [7 8 0]]

Goal State Reached!
